<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
df = pd.read_csv(url, header=None, na_values=['?'])
df.isna().sum().count()


16

In [0]:
df.dropna(inplace=True)

In [56]:
#  convert missing data='?' into NAN, correctly identify feature as numeric from object.
df.describe()


,1,2,7,10,13,14
count,678.000000,690.000000,690.000000,690.00000,677.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.957862,4.978163,3.346513,4.86294,173.806768,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.602500,1.000000,0.165000,0.00000,75.000000,0.000000
50%,28.460000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,38.230000,7.207500,2.625000,3.00000,276.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [57]:
df.describe(exclude=np.number)

,0,3,4,5,6,8,9,11,12,15
count,678,684,684,681,681,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,468,519,519,137,399,361,395,374,625,383


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
# Separate by approve status
df_plus = df[ (df[15] == '+') ]
df_minus= df[ df[15] == '-']

In [121]:
# ttest independent for Approved continuous features
# Any relationship between approved and disapproved continuous features
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel

cont_features_ind = pd.DataFrame(columns=['features','statistic_ind','pvalue_ind'])

i=0
for f in [1,2,7,10,13,14]:
  t_ind, p_ind = ttest_ind(df_plus[f], df_minus[f])
  cont_features_ind.loc[i] = [str(f),t_ind, p_ind]
  i = i + 1
  
cont_features_ind

,features,statistic_ind,pvalue_ind
0,1,4.675662,3.563671e-06
1,2,5.400813,9.310154e-08
2,7,9.002392,2.407924e-18
3,10,11.336965,2.586465e-27
4,13,-2.182219,2.945010e-02
5,14,4.475370,9.003916e-06


In [0]:
df=df.replace('+',1)
df=df.replace('-',0)

In [122]:
# ttest independence test on Approved and Non-approved continuous features
cont_features_ind2 = pd.DataFrame(columns=['features','statistic_ind','pvalue_ind'])

i=0
for f in [1,2,7,10,13,14]:
  t_ind, p_ind = ttest_ind(df[15], df[f])
  cont_features_ind2.loc[i] = [str(f),t_ind, p_ind]
  i = i + 1
  
cont_features_ind2

,features,statistic_ind,pvalue_ind
0,1,-66.965808,0.000000e+00
1,2,-22.137059,1.932402e-92
2,7,-13.430347,1.300827e-38
3,10,-10.485814,9.386469e-25
4,13,-27.316551,2.715155e-130
5,14,-4.929099,9.322324e-07


In [123]:
df.describe()

,1,2,7,10,13,14,15
count,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000
mean,31.503813,4.829533,2.244296,2.502297,180.359877,1013.761103,0.453292
std,11.838267,5.027077,3.371120,4.968497,168.296811,5253.278504,0.498195
min,13.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.580000,1.040000,0.165000,0.000000,73.000000,0.000000,0.000000
50%,28.420000,2.835000,1.000000,0.000000,160.000000,5.000000,0.000000
75%,38.250000,7.500000,2.625000,3.000000,272.000000,400.000000,1.000000
max,76.750000,28.000000,28.500000,67.000000,2000.000000,100000.000000,1.000000


In [125]:
# Ttest dependence for Approved and Non-Approved
for i in [1,2,7,10,13,14]:
  s,p= ttest_rel(df[15],df[i])
  print('Feature category(0,{}) statistic={} and pvalue={}'.format(i,s,p))

Feature category(0,1) statistic=-67.4787207454447 and pvalue=2.540365191314492e-296
Feature category(0,2) statistic=-22.601150021728994 and pvalue=5.768331023080983e-84
Feature category(0,7) statistic=-14.127360654431145 and pvalue=9.963041537198473e-40
Feature category(0,10) statistic=-10.935900684577518 and pvalue=1.1269620554090349e-25
Feature category(0,13) statistic=-27.309662563888832 and pvalue=4.553284777434871e-110
Feature category(0,14) statistic=-4.929179898992901 and pvalue=1.048577025686764e-06


In [124]:
# Testing dependence relationship between feature 1 and 10
stats.ttest_rel(df[15],df[10])

Ttest_relResult(statistic=-10.935900684577518, pvalue=1.1269620554090349e-25)

In [86]:
# Dependence between 2,7
stats.ttest_rel(df[],df[7])

Ttest_relResult(statistic=12.845909604580136, pvalue=7.883639641108741e-34)

In [112]:
# Dependence between 13 and 14
stats.ttest_rel(df[13],df[14])

Ttest_relResult(statistic=26.46938234915939, pvalue=2.0769437865567574e-105)

In [88]:
df.describe(exclude=np.number)

,0,3,4,5,6,8,9,11,12,15
count,653,653,653,653,653,653,653,653,653,653
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,450,499,499,133,381,349,366,351,598,357


In [0]:
from scipy.stats import chisquare

In [106]:
# Applying chisquare for features with two category values
cat_2=pd.crosstab(df[0],[df[8],df[9],df[11]])
s,p=chisquare(cat_2, axis=None)
print('Feature category(0,8,9,11) statistic {} and pvalue {}'.format(s,p))

Feature category(0,8,9,11) statistic 264.24349157733536 and pvalue 1.4328099132249606e-47


In [116]:
for i in [8,9,11]:
  cross= pd.crosstab(df[0],df[i])
  s,p = chisquare(cross, axis=None)
  print('Feature category(0,{}) statistic={} and pvalue={}'.format(i,s,p))

Feature category(0,8) statistic=96.5436447166922 and pvalue=8.600948714043167e-21
Feature category(0,9) statistic=110.2771822358346 and pvalue=9.564956912560129e-24
Feature category(0,11) statistic=97.364471669219 and pvalue=5.7293804384309325e-21


In [108]:
# Applying chisquare for features with three category values
cat_3 = pd.crosstab(df[0],[df[3],df[4],df[12]])
s,p=chisquare(cat_3, axis=None)
print('Feature category(3,4,12): statistic={}, pvalue={}'.format(s,p))

Feature category(3,4,12): statistic=2105.814701378254, pvalue=0.0


In [117]:
for i in [3,4,12]:
  cross = pd.crosstab(df[0],df[i])
  s,p = chisquare(cross, axis=None)
  print('Feature category(0,{}) statistic={} and pvalue={}'.format(i,s,p))


Feature category(0,3) statistic=759.7748851454825 and pvalue=5.814867862621082e-162
Feature category(0,4) statistic=759.7748851454826 and pvalue=5.814867862620749e-162
Feature category(0,12) statistic=1214.3231240428793 and pvalue=2.3199396221271546e-260


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

#### For Continuous Features  with Class Approved  using Ttest independence, ttest_ind
Features 1,2,7,10,14 has pvalue less than 0.01 percent. There are high confidence that  relationship existed between the approved or non-approved cases.

## Feature 13 has pvalue=0.02, greater than 0.01 percent. There is no relationship between approved.

#### For Continuous Features with class Approved and Non-Approved using Ttest indepennce
## Feature 1 has pvalue=0, null hypothesis definitely true. 


### For Continuous features, testing dependency using ttest_rel
Relationship features between {1,10}, {2,7}, {13,14}, all return pvalue < 0.01 .
In fact ttest_rel gives  pvalue<0.01 for all continuous features.
Not sure when to use ttest_rel ?

####  For Categorical features with Class Approved/Non-approved, using ChiSquare
Features {0,8,9,11} has pvalue < 0.01, gives high confidence these features has relationship to approved or non-approved cases.

## Features {3,4,12} has pvalue=0.0, has NO relationship to approved or non-approved cases.
Random chances for approval or non-approval for features {3,4,12}

## Conclusion Result
Feature 1 has relationship with Approved or Not approved cases. Random chance.

Feature {3,4,12} together has no relationship with Approved or Non approved case. Random chance.